Hola voy a empezar a hacer pruebas con dask y regla de simpson, pero si prefieren usar otro metodo o herramienta no tengo problemas

In [24]:
import math
import time
import multiprocessing
import numpy as np
from dask.distributed import Client
from scipy.integrate import quad

In [7]:
f=lambda x: np.exp(-x**2)
a=0
b=1
n=10**6
h_hat=(b-a)/n

In [8]:
def Scf(n,f,a,b):
    """
    Compute numerical approximation using trapezoidal rule in 
    an interval.
    Nodes are generated via numpy
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Tcf (float) 
    """
    h = b-a
    h_hat=h/n
    nodes=[a+(i)*h_hat for i in range(0,n+1)]
    sum_res1=sum(f(np.asarray(nodes[2:n:2])))
    sum_res2=sum(f(np.asarray(nodes[1:n:2])))
    return h/(3*n)*(f(nodes[0])+f(nodes[-1])+2*sum_res1+4*sum_res2)

In [9]:
start_time = time.time()
aprox=Scf(n,f,a,b)
end_time = time.time()

In [14]:
secs = end_time-start_time
print("Scf tomó",secs,"segundos" )

Scf tomó 0.6460535526275635 segundos


In [11]:
obj, err = quad(f, a, b)

In [12]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [13]:
err_relativo(aprox,obj)

6.392347001308277e-15

In [15]:
Rcf_secuencial_timeit = %timeit -n 5 -r 10 -o Scf(n,f,a,b)

649 ms ± 15 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [18]:
p=multiprocessing.cpu_count()
ns_p = int(n/p) 

In [25]:
client = Client()

/home/miuser/.local/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [19]:
def construye_nodos_en_subintervalos(mi_id,a,b,h_hat,n,ns_p):
    begin=mi_id*ns_p
    end=begin+ns_p
    h_hat=(b-a)/n
    nodos = []
    for i in range(begin,end):
        nodos.append(a+(i+1/2.0)*h_hat)
    return nodos

In [20]:
def evalua_f_en_nodos_y_suma(nodos,f):
    suma_res = 0
    for nodo in nodos:
        suma_res+=f(nodo)
    return suma_res

In [26]:
%%time
futures_nodos = client.map(construye_nodos_en_subintervalos,range(p),
                                                            **{'a':a,
                                                               'b':b,
                                                               'h_hat':h_hat,
                                                               'n':n,
                                                               'ns_p':ns_p}
                            )
futures_evalua = client.map(evalua_f_en_nodos_y_suma, futures_nodos,
                                                       **{'f':f}
                            )
results = client.gather(futures_evalua)
aprox=h_hat*sum(results)

CPU times: user 225 ms, sys: 22.2 ms, total: 247 ms
Wall time: 4.19 s


In [27]:
err_relativo(aprox,obj)

4.816559135869493e-14

In [28]:
client.close()